<a href="https://colab.research.google.com/github/KendallScott/QTW/blob/main/Case%20Study%202/Case_Study_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import label_binarize, OneHotEncoder

In [14]:
df = pd.read_csv("https://raw.githubusercontent.com/KendallScott/QTW/main/Case%20Study%202/diabetic_data.csv")
ID_map = pd.read_csv("https://raw.githubusercontent.com/KendallScott/QTW/main/Case%20Study%202/IDs_mapping.csv")

admission_source = ID_map.drop(['admission_description', 'discharge_description'], axis=1)
admission = ID_map.drop(['admission_source_description', 'discharge_description'], axis=1)
discharge = ID_map.drop(['admission_description', 'admission_source_description'], axis=1)

df=pd.merge(df, admission, 
            left_on='admission_type_id', 
            right_on='id', 
            how='left' 
)


df=pd.merge(df, discharge, 
            left_on='discharge_disposition_id', 
            right_on='id', 
            how='left' 
)
df=pd.merge(df, admission_source, 
            left_on='admission_source_id', 
            right_on='id', 
            how='left' 
)

df = df[df.columns.drop(list(df.filter(regex='_x')))]
df = df[df.columns.drop(list(df.filter(regex='_y')))]

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 101766 entries, 0 to 101765
Data columns (total 54 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   encounter_id                  101766 non-null  int64 
 1   patient_nbr                   101766 non-null  int64 
 2   race                          101766 non-null  object
 3   gender                        101766 non-null  object
 4   age                           101766 non-null  object
 5   weight                        101766 non-null  object
 6   admission_type_id             101766 non-null  int64 
 7   discharge_disposition_id      101766 non-null  int64 
 8   admission_source_id           101766 non-null  int64 
 9   time_in_hospital              101766 non-null  int64 
 10  payer_code                    101766 non-null  object
 11  medical_specialty             101766 non-null  object
 12  num_lab_procedures            101766 non-null  int64 
 13 

Upon first inspection there do not appear to be any NAs

In [15]:
df.head()

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted,admission_description,discharge_description,id,admission_source_description
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,NO,NaN,Not Mapped,1,Physician Referral
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,No,No,Ch,Yes,>30,Emergency,Discharged to home,7,Emergency Room
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,Yes,NO,Emergency,Discharged to home,7,Emergency Room
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,No,No,Ch,Yes,NO,Emergency,Discharged to home,7,Emergency Room
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,No,No,Ch,Yes,NO,Emergency,Discharged to home,7,Emergency Room


The weight column has a question mark (?), corresponding to missing weights, these are effectively NAs.
Also, target is not binary in this case, however, our aim is to predict rehospitalization within 30 days, a binary (yes or no) response.

We replaced the "?" values with NaN.

In [16]:
# Replacing the ? with nan
df=df.replace("?", np.nan)

In [17]:
# Get count duplicates single column using dataframe.pivot_table()
df2 = df.pivot_table(index = ['patient_nbr'], aggfunc ='size')
print(df2)

patient_nbr
135          2
378          1
729          1
774          1
927          1
            ..
189351095    1
189365864    1
189445127    1
189481478    1
189502619    1
Length: 71518, dtype: int64


When we look at the data by patient number, we can see that the number of rows drops from 101,766 down to 71,518. We will be dropping the repeated values.

In [18]:
#sorting the dataframe by patient_nbr and encounter_id, in order to filter out the repeat values 
df=df.sort_values(by=['patient_nbr', 'encounter_id'])

#filtering out repeat values by patient_nbr
df=df.groupby('patient_nbr').first()
len(df)

71518

In [19]:
for i in df.columns:
  count = df.loc[df[i]=='?',i].count()
  if count > 0:
    print('{}:'.format(i),
          '{}%'.format(np.round(np.divide(count,len(df))*100,2)))

Three variables have a large percentage of missing values, Weight, Payer_code and Medical_speciatly with about 97%, 40% and 49% of missing values accordingly. Race and diags 1-3 all have a smaller number of missing values.

In [20]:
df[['payer_code','admission_type_id']].groupby('admission_type_id').describe()

payer_code                   
                       count unique  top   freq
admission_type_id                              
1                      22413     16   MC  11780
2                       7779     17   MC   3530
3                       8901     15   MC   4271
4                          4      3   CP      2
5                       2285     12   MC   1459
6                        130     10   MC     84
7                         21      6   CP      8
8                          0      0  NaN    NaN

Admission code 1-Emergency, 2-Urgent, 3-Elective, 4-Newborn, 5-NotAvailable, 6-NULL, 7-Trauma Center, 8-NotMapped.
For Not Mapped and Null admission categories, the missing payer code is seen on almost all patients. Based on the levels of the payer code category, it would seem as if the missing value corresponds to patients who did not disclose a payment method (insurance or self-pay), which in itself, is a category level.

In [21]:
df[['medical_specialty','admission_description']].groupby('admission_description').describe()

medical_specialty                                     
                                  count unique                     top  freq
admission_description                                                       
Elective                           8995     54              Cardiology  1318
Emergency                         15986     59        InternalMedicine  7299
Newborn                               2      2        InternalMedicine     1
Not Available                      1245     22  Family/GeneralPractice   490
Not Mapped                          258     12        InternalMedicine    86
Trauma Center                         0      0                     NaN   NaN
Urgent                             8969     57        Emergency/Trauma  2364

In [28]:
len(df)

71518

In [22]:
df.readmitted.value_counts()
# >30 does not count as within 30 days for our target, will replace

NO     42985
>30    22240
<30     6293
Name: readmitted, dtype: int64

In [23]:
y = df.readmitted.replace(to_replace='>30',value='NO')
y = label_binarize(y,classes=['NO','<30'])

In [24]:
# Make target variable and data variable
X = df.loc[:, df.columns != 'readmitted']

In [25]:
X

,encounter_id,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,...,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,admission_description,discharge_description,id,admission_source_description
patient_nbr,,,,,,,,,,,,,,,,,,,,,
135,24437208,Caucasian,Female,[50-60),None,2,1,1,8,None,...,No,No,No,No,Ch,Yes,Urgent,Discharged to home,1,Physician Referral
378,29758806,Caucasian,Female,[50-60),None,3,1,1,2,None,...,No,No,No,No,No,No,Elective,Discharged to home,1,Physician Referral
729,189899286,Caucasian,Female,[80-90),None,1,3,7,4,MC,...,No,No,No,No,No,Yes,Emergency,Discharged/transferred to SNF,7,Emergency Room
774,64331490,Caucasian,Female,[80-90),None,1,1,7,3,None,...,No,No,No,No,Ch,Yes,Emergency,Discharged to home,7,Emergency Room
927,14824206,AfricanAmerican,Female,[30-40),None,1,1,7,5,None,...,No,No,No,No,No,Yes,Emergency,Discharged to home,7,Emergency Room
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189351095,418513058,Caucasian,Female,[80-90),None,1,1,7,1,None,...,No,No,No,No,No,No,Emergency,Discharged to home,7,Emergency Room
189365864,359719064,Other,Male,[60-70),None,1,1,7,3,HM,...,No,No,No,No,No,Yes,Emergency,Discharged to home,7,Emergency Room
189445127,338462954,Caucasian,Female,[80-90),None,1,1,7,3,None,...,No,No,No,No,Ch,Yes,Emergency,Discharged to home,7,Emergency Room
